<a href="https://colab.research.google.com/github/Rishiatweb/LADEL.ai/blob/main/pipeline/LADEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
print("Installing required libraries")
!pip install -q sentence-transformers scikit-learn pandas numpy pyyaml transformers accelerate torch
print("✅ Dependencies installed successfully.")

Installing required libraries
✅ Dependencies installed successfully.


In [40]:
import os
os.makedirs("ladel", exist_ok=True)
print("Project directory 'ladel/' created.")

Project directory 'ladel/' created.


In [41]:
#to create configuration file
%%writefile config.yaml
##dont touch this if you want to use this pipeline,only to experiment

# --- Embedding Settings ---
embedding_model: 'all-MiniLM-L6-v2' # A fast, good-quality model

# --- Anomaly Detection Settings ---
# You can choose from: 'isolation_forest', 'lof', 'one_class_svm'
detection_model: 'isolation_forest'

isolation_forest:
  n_estimators: 100
  contamination: 0.1 # Expected proportion of anomalies. 'auto' is also an option.
  random_state: 42

local_outlier_factor:
  n_neighbors: 20
  contamination: 0.1
  novelty: True # Must be True to use predict() on new data

one_class_svm:
  kernel: 'rbf'
  nu: 0.1 # An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors. Similar to contamination.

# --- Explanation Settings ---
explanation_enabled: true
llm_model: 'google/flan-t5-base'
max_new_tokens: 256
num_normal_samples_for_context: 10

Overwriting config.yaml


In [49]:
%%writefile ladel/data_loader.py
import os
def load_log_file(filepath: str) -> list[str]:
  """loads a log file, decodes it, and processes it."""
  if not os.path.exists(filepath):
    print(f"Error: File not found at {filepath}")
    return []

  print(f"Reading log file from: {filepath}")
  try:
      with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
          log_content = f.read()

      log_lines = log_content.splitlines()
      processed_lines = [line.strip() for line in log_lines if line.strip()]

      print(f"Read {len(processed_lines)} non-empty log lines.")
      return processed_lines
  except Exception as e:
      print(f"Error reading or processing file: {e}")
      return []

Overwriting ladel/data_loader.py


In [43]:
# Cell 5: Create the Embedding Module
%%writefile ladel/embedding.py
from sentence_transformers import SentenceTransformer
import numpy as np

class LogEmbedder:
    def __init__(self, model_name: str):
        """Initializes the sentence transformer model."""
        self.model_name = model_name
        self.embedder = None
        print(f"\nLoading sentence transformer model: '{self.model_name}'...")
        try:
            self.embedder = SentenceTransformer(self.model_name)
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading embedding model: {e}")

    def encode(self, log_lines: list[str]) -> np.ndarray | None:
        """Generates embeddings for a list of log lines."""
        if not self.embedder or not log_lines:
            print("Embedder not loaded or no log lines to process. Skipping encoding.")
            return None

        print("Generating embeddings for log lines...")
        try:
            embeddings = self.embedder.encode(log_lines, show_progress_bar=True)
            print(f"Generated {embeddings.shape[0]} embeddings with dimension {embeddings.shape[1]}.")
            return embeddings
        except Exception as e:
            print(f"Error during embedding generation: {e}")
            return None

Overwriting ladel/embedding.py


In [44]:
# Cell 6: Create the Anomaly Detection Module
%%writefile ladel/detection.py
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

def get_anomaly_detector(model_name: str, params: dict):
    """Factory function to get an anomaly detection model."""
    print(f"\nInitializing detection model: {model_name}")
    if model_name == 'isolation_forest':
        params['contamination'] = float(params['contamination']) if str(params['contamination']) != 'auto' else 'auto'
        params['n_estimators'] = int(params['n_estimators'])
        return IsolationForest(**params)
    elif model_name == 'lof':
        params['n_neighbors'] = int(params['n_neighbors'])
        params['contamination'] = float(params['contamination'])
        return LocalOutlierFactor(**params)
    elif model_name == 'one_class_svm':
        params['nu'] = float(params['nu'])
        return OneClassSVM(**params)
    else:
        raise ValueError(f"Unknown detection model: {model_name}")

def train_and_predict(model, embeddings: np.ndarray) -> tuple[np.ndarray | None, np.ndarray | None]:
    """Trains the model and returns predictions and indices of anomalies."""
    if embeddings is None or embeddings.shape[0] == 0:
        print("No embeddings available. Skipping anomaly detection.")
        return None, None

    print(f"Training {model.__class__.__name__} for anomaly detection...")
    try:
        # LocalOutlierFactor uses fit_predict for training data
        if isinstance(model, LocalOutlierFactor):
            predictions = model.fit_predict(embeddings)
        else:
            model.fit(embeddings)
            predictions = model.predict(embeddings)

        anomalous_indices = np.where(predictions == -1)[0]
        print(f"Model identified {len(anomalous_indices)} potential anomalies.")
        return predictions, anomalous_indices
    except Exception as e:
        print(f"Error during model training or prediction: {e}")
        return None, None

Overwriting ladel/detection.py


In [51]:
# Cell 7: Create the Explanation Module
%%writefile ladel/explanation.py
from transformers import pipeline, logging
import torch
import textwrap

# Suppress verbose logging from transformers to keep output clean
logging.set_verbosity_error()

class LogExplainer:
    def __init__(self, model_name: str):
        """Initializes the text-generation pipeline for explanations."""
        self.generator = None
        print("\nSetting up Hugging Face text-generation pipeline...")
        device = 0 if torch.cuda.is_available() else -1
        if device == 0:
            print(f"GPU found. Loading model '{model_name}' to GPU.")
        else:
            print(f"GPU not available. Loading model '{model_name}' to CPU (will be slow).")

        try:
            self.generator = pipeline('text2text-generation', model=model_name, device=device)
            print("LLM explainer loaded successfully!")
        except Exception as e:
            print(f"Error loading LLM explainer model: {e}")

    def generate_explanation(self, anomaly_log: str, normal_logs_sample: list[str], max_new_tokens: int = 128) -> str:
        """Generates a concise explanation for a single anomaly."""
        if not self.generator:
            return "LLM explainer not available."

        normal_logs_text = "\n".join(normal_logs_sample)
        prompt_template = """
### INSTRUCTIONS ###
You are an expert log analysis assistant. Your task is to explain why the "[ANOMALOUS log]" is an anomaly.
To do this, first observe the pattern in the "[NORMAL logs]" which represent typical behavior.
Then, contrast the "[ANOMALOUS log]" with this typical behavior, highlighting the key difference that makes it an anomaly.
Provide your analysis under "[ANALYSIS]". Be specific and concise.

### EXAMPLE OF GOOD ANALYSIS ###
[NORMAL logs]
2023-11-15 10:00:05 INFO: User 'alice' logged in successfully.
2023-11-15 10:01:12 INFO: User 'bob' accessed the dashboard.
2023-11-15 10:02:00 INFO: User 'charlie' updated their profile.

[ANOMALOUS log]
2023-11-15 10:03:45 CRITICAL: Database connection failed: timeout expired.

[ANALYSIS]
The normal logs show routine user activities such as logins and profile updates, all marked with INFO severity.
The anomalous log, however, reports a CRITICAL system-level failure: a database connection timeout.
This is an anomaly because it indicates a severe service disruption, differing in both severity (CRITICAL vs. INFO) and event type (system failure vs. user action) from the typical logs.

### YOUR TASK ###
[NORMAL logs]
{normal_logs_text}

[ANOMALOUS log]
{anomaly_log_text}

[ANALYSIS]
"""
        final_prompt = prompt_template.format(normal_logs_text=normal_logs_text, anomaly_log_text=anomaly_log)

        try:
            outputs = self.generator(final_prompt, max_new_tokens=max_new_tokens)
            return textwrap.fill(outputs[0]['generated_text'], width=100)
        except Exception as e:
            return f"Error during explanation generation: {e}"

Overwriting ladel/explanation.py


In [46]:
# Cell 8: Create the Main Pipeline Script
%%writefile ladel/main.py
import argparse
import yaml
import numpy as np
import random
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

# Import our custom modules
from ladel.data_loader import load_log_file
from ladel.embedding import LogEmbedder
from ladel.detection import get_anomaly_detector, train_and_predict
from ladel.explanation import LogExplainer

def run_pipeline(config_path: str, log_file_path: str):
    """Executes the full log anomaly detection and explanation pipeline."""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print("--- Configuration Loaded ---")
    print(f"Detection Model: {config['detection_model']}")

    log_lines = load_log_file(log_file_path)
    if not log_lines:
        print("No log lines to process. Exiting.")
        return

    embedder = LogEmbedder(model_name=config['embedding_model'])
    log_embeddings = embedder.encode(log_lines)

    detector_name = config['detection_model']
    detector_params = config[detector_name]
    detector = get_anomaly_detector(detector_name, detector_params)
    predictions, anomalous_indices = train_and_predict(detector, log_embeddings)

    if predictions is None:
        print("Anomaly detection failed. Cannot proceed.")
        return

    print("\n" + "="*25 + " RESULTS " + "="*25)
    normal_indices = np.where(predictions == 1)[0]
    print(f"Total lines processed: {len(log_lines)}")
    print(f"Normal lines found: {len(normal_indices)}")
    print(f"Potential anomalies detected: {len(anomalous_indices)}")

    if len(anomalous_indices) == 0:
        print("\nNo anomalies found. All logs appear to be normal.")
        return

    print("\n--- Anomalous Logs ---")
    for idx in anomalous_indices:
        print(f"[ANOMALY] Line {idx+1}: {log_lines[idx]}")

    if config.get('explanation_enabled', False):
        explainer = LogExplainer(model_name=config['llm_model'])
        if explainer.generator:
            print("\n" + "="*20 + " LLM EXPLANATIONS " + "="*20)
            sample_size = min(config['num_normal_samples_for_context'], len(normal_indices))
            if sample_size > 0:
                sample_normal_indices = random.sample(list(normal_indices), sample_size)
                normal_log_samples = [log_lines[i] for i in sample_normal_indices]
                for i, anomaly_idx in enumerate(anomalous_indices):
                    anomaly_log = log_lines[anomaly_idx]
                    print(f"\n--- Analysis for Anomaly #{i+1} (Line {anomaly_idx+1}) ---")
                    print(f"Anomalous Log: `{anomaly_log}`")
                    explanation = explainer.generate_explanation(anomaly_log, normal_log_samples)
                    print("\n[EXPLANATION]")
                    print(explanation)
            else:
                print("\nCannot generate LLM explanations: No 'normal' logs found for baseline.")

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Log Anomaly Detection Pipeline")
    parser.add_argument("--config", type=str, default="config.yaml", help="Path to config file.")
    parser.add_argument("--log_file", type=str, default="app.log", help="Path to log file.")
    args = parser.parse_args()
    run_pipeline(config_path=args.config, log_file_path=args.log_file)



Overwriting ladel/main.py


In [53]:
# Cell 9: Create Sample Log File for Testing
%%writefile app.log
2023-10-26 10:00:01 INFO: Application startup initiated. Version 1.2.3
2023-10-26 10:00:02 INFO: Configuration loaded from /etc/app/config.xml
2023-10-26 10:00:03 DEBUG: Database connection pool initialized with 10 connections.
2023-10-26 10:00:04 INFO: Listening on port 8080
2023-10-26 10:01:15 INFO: User 'alice' logged in successfully from IP 192.168.1.10
2023-10-26 10:01:17 DEBUG: User 'alice' session created: session_abc123
2023-10-26 10:01:20 INFO: Request received: GET /api/data?id=123 user='alice'
2023-10-26 10:01:21 DEBUG: Querying database for item id=123
2023-10-26 10:01:22 INFO: Data retrieval successful for item id=123. Rows: 1
2023-10-26 10:01:25 INFO: User 'bob' logged in successfully from IP 192.168.1.12
2023-10-26 10:01:27 DEBUG: User 'bob' session created: session_def456
2023-10-26 10:02:00 INFO: Request received: POST /api/update user='alice'
2023-10-26 10:02:01 DEBUG: Validating input data for update.
2023-10-26 10:02:02 INFO: Data update successful for user 'alice'.
2023-10-26 10:02:30 WARN: API rate limit approaching for user 'bob'. 950/1000 requests.
2023-10-26 10:03:00 INFO: Scheduled job 'daily_backup' started.
2023-10-26 10:03:05 INFO: Request received: GET /api/data?id=456 user='bob'
2023-10-26 10:03:06 DEBUG: Querying database for item id=456
2023-10-26 10:03:07 INFO: Data retrieval successful for item id=456. Rows: 1
2023-10-26 10:03:10 ERROR: Failed to process payment for order_789. Reason: Insufficient funds. Customer: CUST007
2023-10-26 10:03:12 DEBUG: Payment failure logged for order_789
2023-10-26 10:04:00 INFO: User 'charlie' logged in successfully from IP 192.168.1.15
2023-10-26 10:04:02 DEBUG: User 'charlie' session created: session_ghi789
2023-10-26 10:04:05 INFO: Request received: GET /api/data?id=789 user='charlie'
2023-10-26 10:04:06 DEBUG: Querying database for item id=789
2023-10-26 10:04:07 INFO: Data retrieval successful for item id=789. Rows: 1
2023-10-26 10:05:00 INFO: User 'alice' logged out. Session: session_abc123
2023-10-26 10:05:15 WARN: High memory usage detected: 85% used.
2023-10-26 10:05:18 INFO: Request received: DELETE /api/resource/xyz user='admin_priv'
2023-10-26 10:05:19 INFO: Resource 'xyz' deleted successfully by 'admin_priv'.
2023-10-26 10:06:00 INFO: User 'dave' attempted login from IP 203.0.113.45 - FAILED (Invalid Credentials)
2023-10-26 10:06:01 INFO: User 'dave' attempted login from IP 203.0.113.45 - FAILED (Invalid Credentials)
2023-10-26 10:06:02 INFO: User 'dave' attempted login from IP 203.0.113.45 - FAILED (Invalid Credentials)
2023-10-26 10:06:03 INFO: User 'dave' attempted login from IP 203.0.113.45 - FAILED (Invalid Credentials)
2023-10-26 10:06:04 WARN: Multiple failed login attempts for user 'dave' from IP 203.0.113.45. Account locked temporarily.
2023-10-26 10:07:00 INFO: Request received: GET /api/system_health user='monitor_agent'
2023-10-26 10:07:01 INFO: System health check: OK. CPU: 30%, Mem: 60%, Disk: 40%
2023-10-26 10:08:00 INFO: Request received: GET /api/data?id=000 user='bob'
2023-10-26 10:08:01 DEBUG: Querying database for item id=000
2023-10-26 10:08:02 INFO: Data retrieval successful for item id=000. Rows: 1
2023-10-26 10:09:00 INFO: Scheduled job 'data_cleanup' started.
2023-10-26 10:10:00 INFO: User 'eve_hacker' logged in successfully from IP 10.0.0.5 (Internal Test Account)
2023-10-26 10:10:05 INFO: Request received: GET /admin/config_dump user='eve_hacker'
2023-10-26 10:10:06 CRITICAL: Unauthorized access attempt to /admin/config_dump by user 'eve_hacker' from IP 10.0.0.5.
2023-10-26 10:10:07 INFO: User 'eve_hacker' session terminated. IP 10.0.0.5 blocked.
2023-10-26 10:11:00 INFO: Data processing batch 'batch_alpha' started. Records: 10000
2023-10-26 10:11:05 DEBUG: Processing record 1 of 10000 in 'batch_alpha'
2023-10-26 10:11:10 DEBUG: Processing record 500 of 10000 in 'batch_alpha'
2023-10-26 10:11:15 ERROR: Timeout while connecting to external service 'payment_gateway_v2'. URL: https://api.payments.example.com/charge
2023-10-26 10:11:17 WARN: Retrying connection to 'payment_gateway_v2' (Attempt 1/3)
2023-10-26 10:11:20 ERROR: Timeout while connecting to external service 'payment_gateway_v2'. URL: https://api.payments.example.com/charge
2023-10-26 10:11:22 WARN: Retrying connection to 'payment_gateway_v2' (Attempt 2/3)
2023-10-26 10:11:25 ERROR: Timeout while connecting to external service 'payment_gateway_v2'. URL: https://api.payments.example.com/charge
2023-10-26 10:11:27 FATAL: Failed to connect to 'payment_gateway_v2' after 3 retries. Aborting batch 'batch_alpha'.
2023-10-26 10:11:30 INFO: Data processing batch 'batch_alpha' failed.
2023-10-26 10:12:00 INFO: User 'frank' logged in successfully from IP 192.168.2.22
2023-10-26 10:12:05 INFO: Request to deprecated endpoint /api/v1/status by user 'frank'.
2023-10-26 10:13:00 INFO: New feature flag 'beta_feature_X' enabled for user 'alice'.
2023-10-26 10:14:00 INFO: Database schema migration version 3.4.1 started.
2023-10-26 10:14:30 INFO: Database schema migration version 3.4.1 completed successfully.
2023-10-26 10:15:00 INFO: System maintenance window starting in 60 minutes.
2023-10-26 10:15:01 INFO: User 'grace' logged in successfully from IP 192.168.3.33
2023-10-26 10:15:05 DEBUG: User 'grace' accessing /dashboard
2023-10-26 10:15:10 INFO: Report 'monthly_sales' generated. Size: 2.5MB
2023-10-26 10:16:00 INFO: Unexpected input format for field 'user_preference'. Value: '{"theme": "dark mode"}' instead of 'dark'. User: 'bob'
2023-10-26 10:16:05 DEBUG: Attempting to parse 'user_preference' with fallback.
2023-10-26 10:17:00 INFO: Email sent to admin@example.com: System CPU usage high
2023-10-26 10:18:00 INFO: User 'heidi' logged in successfully from IP 192.168.1.18
2023-10-26 10:18:05 DEBUG: Processing background task: image_resize_job_999
2023-10-26 10:18:10 INFO: Background task 'image_resize_job_999' completed. Output: /path/to/resized_img.jpg
2023-10-26 10:19:00 SEVERE: Core component 'MessageQueue' unresponsive. All queue operations paused.
2023-10-26 10:19:05 INFO: Attempting to restart 'MessageQueue' component.
2023-10-26 10:19:10 INFO: 'MessageQueue' component restarted successfully.
2023-10-26 10:20:00 INFO: User 'ivan' logged in successfully from IP 172.16.0.5
2023-10-26 10:20:05 INFO: File /tmp/big_upload.dat received. Size: 1024MB. Processing...
2023-10-26 10:21:00 WARN: Disk space on /tmp is now 95% full. Cleaning up old files.
2023-10-26 10:21:05 INFO: Old files in /tmp cleaned. Disk space at 70%.
2023-10-26 10:22:00 INFO: User 'judy' logged in successfully from IP 192.168.4.50
2023-10-26 10:22:05 DEBUG: User 'judy' initiated data export. Format: CSV
2023-10-26 10:22:10 INFO: Data export for 'judy' (all_transactions.csv) completed.
2023-10-26 10:23:00 INFO: Security audit: User 'admin_root' accessed sensitive data table 'user_credentials'. Justification: Scheduled audit.
2023-10-26 10:24:00 INFO: Application performing self-test. All systems nominal.
2023-10-26 10:25:00 INFO: User 'bob' updated profile. Changed email.
2023-10-26 10:25:05 DEBUG: Sending email confirmation to bob_new_email@example.com
2023-10-26 10:26:00 INFO: External API call to 'weather_service' successful. Temp: 15C
2023-10-26 10:27:00 ERROR: Unhandled Python exception: KeyError 'missing_field' in module 'data_processor.py' line 245
2023-10-26 10:27:01 DEBUG: Stack trace for KeyError: ... (omitted for brevity) ...
2023-10-26 10:28:00 INFO: User 'ken' logged in successfully from IP 192.168.1.30
2023-10-26 10:28:05 INFO: A peculiar cosmic ray event was detected by the server's internal chronometer. Time may be temporarily distorted.
2023-10-26 10:29:00 INFO: Service 'recommendation_engine' reloaded with new model version 2.5.
2023-10-26 10:30:00 INFO: All services healthy. Application running normally.
2023-10-26 10:30:01 INFO: User 'laura' logged in successfully from IP 192.168.5.60
2023-10-26 10:30:05 DEBUG: User 'laura' viewing product page 'prod_BXT7'
2023-10-26 10:30:10 INFO: Added item 'prod_BXT7' to cart for user 'laura'
2023-10-26 10:31:00 WARN: Certificate for 'external.partner.api.com' expiring in 7 days.
2023-10-26 10:32:00 INFO: User 'mike' logged in successfully from IP 10.10.10.10
2023-10-26 10:32:05 DEBUG: User 'mike' searching for 'rare_item_name'
2023-10-26 10:32:10 INFO: Search for 'rare_item_name' yielded 0 results.
2023-10-26 10:33:00 INFO: Resource allocation for 'batch_beta' increased. CPU: 4, Mem: 16GB
2023-10-26 10:33:05 INFO: Data processing batch 'batch_beta' started. Records: 500
2023-10-26 10:33:10 DEBUG: Processing record 1 of 500 in 'batch_beta'
2023-10-26 10:34:00 INFO: New user registered: 'user_newbie_001'. Welcome email sent.
2023-10-26 10:35:00 INFO: Database connection pool health: 8/10 connections active.
2023-10-26 10:36:00 INFO: User 'bob' logged out. Session: session_def456
2023-10-26 10:37:00 INFO: Request received: GET /api/data?id=XYZ user='charlie'
2023-10-26 10:37:01 DEBUG: Querying database for item id=XYZ
2023-10-26 10:37:02 INFO: Data retrieval successful for item id=XYZ. Rows: 1
2023-10-26 10:38:00 INFO: System reboot scheduled for 2023-10-27 02:00:00 UTC.
2023-10-26 10:39:00 INFO: User 'admin_ops' initiated manual cache flush.
2023-10-26 10:39:05 INFO: Cache flush completed.
2023-10-26 10:40:00 INFO: All good, nothing to report, just chilling.
2023-10-26 10:40:01 INFO: Everything is perfectly fine and normal. Continue operations.
2023-10-26 10:40:02 INFO: This is a standard operational message, number 345.
2023-10-26 10:40:03 INFO: Processing complete for user 'zeta', all systems green.
2023-10-26 10:40:04 INFO: Final log entry for this batch, signing off.
2023-10-26 10:41:00 ERROR: Corrupted data packet received from sensor SENSOR_003. Data: 0xDEADBEEF...
2023-10-26 10:42:00 INFO: Service 'user_auth' restarting due to minor glitch.
2023-10-26 10:42:05 INFO: Service 'user_auth' back online.
2023-10-26 10:43:00 WARN: API version v1 for /api/data is deprecated. Advise clients to use v2.
2023-10-26 10:44:00 INFO: Performing routine data integrity check...
2023-10-26 10:44:30 INFO: Data integrity check passed. No issues found.
2023-10-26 10:45:00 INFO: Application shutting down gracefully. Goodbye!

Overwriting app.log


In [55]:
# Cell 10: Run the Analysis Pipeline
print("EXECUTING LOG ANALYSIS PIPELINE")
# We run the main script. It will automatically use 'config.yaml' and 'app.log'.
!python ladel/main.py

EXECUTING LOG ANALYSIS PIPELINE
2025-06-10 14:25:39.347819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749565539.382524   19491 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749565539.392688   19491 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 14:25:39.423989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
--- Configuration Loaded ---
Detection Model: isolation_forest
Reading log file from: app.log